In [1]:
# messy imports to be tidied up later
import os
import xlutils, xypath
import databaker
import databaker.constants
from databaker.constants import *
import databaker.databakersolo as ds
from databaker.jupybakeutils import savepreviewhtml, savepreviewhtmlBAGS, writetechnicalCSV, procrows, TechnicalCSV, yield_dimension_values, DUPgenerate_header_row
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [2]:
"""
bake --preview ILCH.py ilchtablestemplatensa.xls growth
bake --preview ILCH.py ilchtablestemplatensa.xls level

bake --preview ILCH.py ilchtablestemplatesa.xls growth
bake --preview ILCH.py ilchtablestemplatesa.xls level
"""

# input-output
inputfile = "ilchtablestemplatesa.xls"
outputfile = "%s.csv" % os.path.splitext(inputfile)[0]
previewfile = "preview.html"


In [3]:
print("Loading %s which has size %d bytes" % (inputfile, os.path.getsize(inputfile)))
tableset = xypath.loader.table_set(inputfile, extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names: %s" % ", ".join([tab.name  for tab in tabs]))


Loading ilchtablestemplatesa.xls which has size 407552 bytes
Table names: INTRODUCTION, DEFINITIONS, 1. Industry level SA, 2. Sector level SA, 3. Industry annual growth SA, 4. Sector annual growth SA, 5. Industry quarterly growth SA, 6. Sector quarterly growth SA


In [4]:
savepreviewhtmlBAGS(tabs[2], previewfile)

opening file preview.html
table '1. Industry level SA' written


In [5]:
newtabs = []
for tab in tabs:
    tab_title = tab.excel_ref("A1")
    if tab_title.filter(contains_string("growth")):    
        newtabs.append(tab)
tab = newtabs[0]
#[tab.name  for tab in newtabs]

In [4]:
conversionsegments = [ ]
for tab in tabs:
    tab_title = tab.excel_ref("A1")
    if not tab_title.filter(contains_string("growth")):    # this is the word that was in PARAMS(1)
        continue
        
    anchor = tab.filter("Period").assert_one()

    datarows = anchor.fill(DOWN).is_not_blank()
    datacols = anchor.shift(DOWN).fill(RIGHT).is_not_blank()
    
    # !!! this is where we are rewriting the values directly to strip out the bad names
    for c in datarows:
        v = c._cell.value
        c._cell.value = v[:6]
        print(c, "changed from", v)

    obs = datarows.waffle(datacols).is_not_blank()
    savepreviewhtmlBAGS(obs, previewfile)

    dimensions = [ 
        (datarows, TIME, DIRECTLY, LEFT), 
        (datacols.parent(), "Costs", DIRECTLY, ABOVE), 
        (anchor.fill(RIGHT).parent(), "SIC", CLOSEST, LEFT)
    ]

    if tab_title.filter(contains_string("year on year")):
        dimensions.append(("Annual", "Growth Period", None, None))
        dimensions.append(("Percent", MEASURETYPE, None, None))
    elif tab_title.filter(contains_string("quarter on quarter")):
        dimensions.append(("Quarterly", "Growth Period", None, None))
        dimensions.append(("Percent", MEASURETYPE, None, None))
    elif tab_title.filter(contains_string("growth rates")):
        dimensions.append(("Annual", "Growth Period", None, None))
        dimensions.append(("Percent", MEASURETYPE, None, None))
    else:
        assert 0 #tab.dimension(MEASURETYPE,"Index")
    if tab_title.filter(contains_string("Non-Seasonally")):
        dimensions.append(("Not seasonally adjusted", "SA / NSA", None, None))
    else:
        dimensions.append(("Seasonally adjusted", "SA / NSA", None, None))

    conversionsegment = (tab, dimensions, obs)
    conversionsegments.append(conversionsegment)
    
savepreviewhtml(conversionsegments[-1], batchcelllookup, previewfile)


{<A7 '2000Q1'>} changed from 2000Q1
{<A8 '2000Q2'>} changed from 2000Q2
{<A9 '2000Q3'>} changed from 2000Q3
{<A10 '2000Q4'>} changed from 2000Q4
{<A11 '2001Q1'>} changed from 2001Q1
{<A12 '2001Q2'>} changed from 2001Q2
{<A13 '2001Q3'>} changed from 2001Q3
{<A14 '2001Q4'>} changed from 2001Q4
{<A15 '2002Q1'>} changed from 2002Q1
{<A16 '2002Q2'>} changed from 2002Q2
{<A17 '2002Q3'>} changed from 2002Q3
{<A18 '2002Q4'>} changed from 2002Q4
{<A19 '2003Q1'>} changed from 2003Q1
{<A20 '2003Q2'>} changed from 2003Q2
{<A21 '2003Q3'>} changed from 2003Q3
{<A22 '2003Q4'>} changed from 2003Q4
{<A23 '2004Q1'>} changed from 2004Q1
{<A24 '2004Q2'>} changed from 2004Q2
{<A25 '2004Q3'>} changed from 2004Q3
{<A26 '2004Q4'>} changed from 2004Q4
{<A27 '2005Q1'>} changed from 2005Q1
{<A28 '2005Q2'>} changed from 2005Q2
{<A29 '2005Q3'>} changed from 2005Q3
{<A30 '2005Q4'>} changed from 2005Q4
{<A31 '2006Q1'>} changed from 2006Q1
{<A32 '2006Q2'>} changed from 2006Q2
{<A33 '2006Q3'>} changed from 2006Q3
{<A3

In [6]:
len(conversionsegments)

4

In [6]:
savepreviewhtml(conversionsegments[-1], batchcelllookup, previewfile)   # <---- preview thing

opening file preview.html
table '6. Sector quarterly growth SA' written
javascript calculated


In [5]:
print("converting and writing %d conversion segments into %s" % (len(conversionsegments), outputfile))
convertedrows = [ ]
for conversionsegment in conversionsegments:
    print("conversion segment size %d" % len(conversionsegment[2]))
    rows = procrows(conversionsegment, batchcelllookup)
    convertedrows.append(rows)
    
# Should detect errors and throw up preview with cells marked
# (though actually the errors manifest further upstream, normally)
    
# here we can do further sorting and postprocessing and allocating to different files 


converting and writing 4 conversion segments into ilchtablestemplatesa.csv
conversion segment size 5952
conversion segment size 2232
conversion segment size 6240
conversion segment size 2340


In [8]:

writetechnicalCSV(outputfile, conversionsegments, convertedrows)



writing 4 conversion segments into ilchtablestemplatesa.csv
conversionwrite segment size 5952
conversionwrite segment size 2232
conversionwrite segment size 6240
conversionwrite segment size 2340
